In [2]:
import numpy as np
import cvxpy as cp
from utils import compute_PTDF,parse_m_file


np.set_printoptions(precision=3, suppress=True, floatmode='fixed')


In [7]:

# Parameters
num_scenarios = 10
perturbation_scale = 0.3
baseMVA = 100

bus_data, gen_data, branch_data, gencost_data = parse_m_file('case14.m')
num_buses = bus_data.shape[0]
num_generators = gen_data.shape[0]
num_lines = branch_data.shape[0]
num_branches = branch_data.shape[0]

# Data Extraction
Pd_base = bus_data[:, 2]
Pg_min = gen_data[:, 9]
Pg_max = gen_data[:, 8]
# Pg_max[0] = 400
# Pg_max[1] = 100
cost_coeff_true = gencost_data[:, 5]

branch_data_congested = branch_data.copy()
#branch_data_congested[:, 5] *= 0.7  # reduce limits by 30%

# PTDF = compute_PTDF(branch_data, bus_data)
PTDF = compute_PTDF(branch_data_congested, bus_data)


# Generator incidence matrix
gen_to_bus = np.zeros((num_buses, num_generators))
for i, gen_bus in enumerate(gen_data[:, 0].astype(int) - 1):
    gen_to_bus[gen_bus, i] = 1

# Storage for scenario data
Pg_scenarios = []
lambda_slack_scenarios = []
nu_max_scenarios = []
nu_min_scenarios = []
mu_min_scenarios = []
mu_max_scenarios = []
Pd_scenarios = []

scenario_count = 0
np.random.seed(42)

while scenario_count < num_scenarios:
    Pd_perturbed = Pd_base * (1 + np.random.uniform(-perturbation_scale, perturbation_scale, size=num_buses))
    #Pd_perturbed = np.random.randint(0, 100, size=Pd_base.shape) + np.random.randint(0, 100, size=Pd_base.shape)/100
    Pd_perturbed[0] = 0  # slack bus
    Pd_scenarios.append(Pd_perturbed)
    
    Pg = cp.Variable(num_generators)
    P_inj = gen_to_bus @ Pg - Pd_perturbed
    P_inj_reduced = P_inj[1:]

    constraints = [
        cp.sum(Pg) == np.sum(Pd_perturbed),
        Pg >= Pg_min,
        Pg <= Pg_max,
        -branch_data_congested[:, 5] <= PTDF @ P_inj_reduced,
        PTDF @ P_inj_reduced <= branch_data_congested[:, 5]
    ]

    objective = cp.Minimize(cp.sum(cp.multiply(cost_coeff_true, Pg)))
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)

    if prob.status == 'optimal':
        scenario_count += 1
        Pg_scenarios.append(Pg.value)
        Pd_scenarios.append(Pd_perturbed)
        lambda_slack_scenarios.append(constraints[0].dual_value)
        nu_min_scenarios.append(constraints[3].dual_value)
        nu_max_scenarios.append(constraints[4].dual_value)
        mu_min_scenarios.append(constraints[1].dual_value)
        mu_max_scenarios.append(constraints[2].dual_value)
        print(f"Scenario {scenario_count}/{num_scenarios} generated.")

Pg_scenarios = np.array(Pg_scenarios)
lambda_slack_scenarios = np.array(lambda_slack_scenarios)
nu_max_scenarios = np.array(nu_max_scenarios)
nu_min_scenarios = np.array(nu_min_scenarios)
mu_min_scenarios = np.array(mu_min_scenarios)
mu_max_scenarios = np.array(mu_max_scenarios)



Scenario 1/10 generated.
Scenario 2/10 generated.
Scenario 3/10 generated.
Scenario 4/10 generated.
Scenario 5/10 generated.
Scenario 6/10 generated.
Scenario 7/10 generated.
Scenario 8/10 generated.
Scenario 9/10 generated.
Scenario 10/10 generated.


In [8]:
print("lambda_slack_scenarios:\n", lambda_slack_scenarios)
print("Pg_scenarios:\n", Pg_scenarios)
print("Pd_scenarios:\n", Pd_scenarios)
print("nu_max_scenarios:\n", nu_max_scenarios)
print("nu_min_scenarios:\n", nu_min_scenarios)  

lambda_slack_scenarios:
 [-7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921 -7.921]
Pg_scenarios:
 [[280.677   0.000  -0.000  -0.000   0.000]
 [237.056   0.000  -0.000  -0.000   0.000]
 [242.189   0.000  -0.000  -0.000   0.000]
 [271.363   0.000  -0.000  -0.000   0.000]
 [219.896   0.000  -0.000  -0.000   0.000]
 [230.022   0.000  -0.000  -0.000   0.000]
 [280.255   0.000  -0.000  -0.000   0.000]
 [223.479   0.000  -0.000  -0.000   0.000]
 [283.399   0.000  -0.000  -0.000   0.000]
 [244.674   0.000  -0.000  -0.000   0.000]]
Pd_scenarios:
 [array([  0.000,  27.568, 107.312,  50.630,   6.031,   8.888,   0.000,
         0.000,  31.290,  10.124,   2.493,   7.820,  16.193,  12.328]), array([  0.000,  27.568, 107.312,  50.630,   6.031,   8.888,   0.000,
         0.000,  31.290,  10.124,   2.493,   7.820,  16.193,  12.328]), array([ 0.000, 17.578, 83.136, 48.510,  7.290,  9.797,  0.000,  0.000,
       25.821,  8.278,  3.408,  7.144, 11.067, 15.027]), array([ 0.000, 17.578, 83.136

In [9]:
# Inverse Optimization (using multiple scenarios)
coef = cp.Variable(num_generators, nonneg=True)
loss = 0
BigM = 1e6

Pg_inv = cp.Variable((num_scenarios, num_generators))
lambda_slack_inv = cp.Variable(num_scenarios)   
nu_min_inv = cp.Variable((num_scenarios, num_lines))
nu_max_inv = cp.Variable((num_scenarios, num_lines))
mu_min_inv = cp.Variable((num_scenarios, num_generators))
mu_max_inv = cp.Variable((num_scenarios, num_generators))
epsilon_line = cp.Variable((num_scenarios, num_lines))
epsilon_gen = cp.Variable((num_scenarios, num_generators))

constraints_inv = []
stationarity = []
primal_feasibility = []
dual_feasibility = []
complementary = []

for t in range(num_scenarios):
    
    P_inj_inv = gen_to_bus @ Pg_inv[t] - Pd_scenarios[t]
    P_inj_reduced_inv = P_inj_inv[1:]
    flow_lines = PTDF @ P_inj_reduced_inv
    
    for gen in range(num_generators):
        # stationarity condition for generator
        stationarity.append(
            coef[gen] + lambda_slack_inv[t] + mu_max_inv[t][gen] - mu_min_inv[t][gen]      
                    + (nu_max_inv[t] - nu_min_inv[t]) @ PTDF @ gen_to_bus[1:, gen] == 0
        )
    
    primal_feasibility += [
        cp.sum(Pg_inv[t]) == np.sum(Pd_scenarios[t]),
        Pg_inv[t] >= Pg_min,
        Pg_inv[t] <= Pg_max,
        -branch_data[:, 5] <= flow_lines,
        flow_lines <= branch_data[:, 5]
    ]

    dual_feasibility += [
        mu_min_inv >= 0,
        mu_max_inv >= 0,
        nu_min_inv >= 0,
        nu_max_inv >= 0
    ]
    
    # Complementary slackness conditions
    for i in range(num_branches):
        # nu_min * (- branch_data_congested[:, 5] - PTDF @ P_inj_reduced) == 0
        if nu_min_scenarios[t][i] != 0:
            complementary.append(cp.abs(-branch_data_congested[i, 5] - flow_lines[i])<= epsilon_line[t][i])
            dual_feasibility.append(nu_max_inv[t][i] <= epsilon_line[t][i]) # check this
        # nu_max * (PTDF @ P_inj_reduced - branch_data_congested[:, 5]) == 0 
        elif nu_max_scenarios[t][i] != 0:
            complementary.append(cp.abs(flow_lines[i] - branch_data_congested[i, 5])<= epsilon_line[t][i])
            dual_feasibility.append(nu_min_inv[t][i] <= epsilon_line[t][i]) # check this
        else:     
            complementary.append(nu_min_inv[t][i] <= epsilon_line[t][i])
            complementary.append(nu_max_inv[t][i] <= epsilon_line[t][i])
           
    
    for i in range(num_generators):
        # mu_max * (Pg - Pg_max) == 0
        if Pg_scenarios[t][i] == Pg_max[i]:
            dual_feasibility.append(mu_max_inv[t][i] >= 0)
            dual_feasibility.append(mu_min_inv[t][i] <= epsilon_gen[t][i])
        # mu_min * (Pg_min - Pg) == 0
        elif Pg_scenarios[t][i] == Pg_min[i]:
            dual_feasibility.append(mu_min_inv[t][i] >= 0)
            dual_feasibility.append(mu_max_inv[t][i] <= epsilon_gen[t][i])
        else:
            dual_feasibility.append(mu_max_inv[t][i] <= epsilon_gen[t][i])
            dual_feasibility.append(mu_min_inv[t][i] <= epsilon_gen[t][i])
    
    # Loss function
    loss += cp.norm(Pg_inv[t] - Pg_scenarios[t], 2)**2
    loss += cp.norm(lambda_slack_inv[t] - lambda_slack_scenarios[t], 2)**2
    for i in range(num_generators):
        loss += cp.norm(mu_min_inv[t][i] - mu_min_scenarios[t][i], 2)**2
        loss += cp.norm(mu_max_inv[t][i] - mu_max_scenarios[t][i], 2)**2
        loss += epsilon_gen[t][i] * BigM
    for i in range(num_lines):      
        loss += cp.norm(nu_max_inv[t][i] - nu_max_scenarios[t][i], 2)**2
        loss += cp.norm(nu_min_inv[t][i] - nu_min_scenarios[t][i], 2)**2
        loss += epsilon_line[t][i] * BigM

constraints_inv += stationarity + primal_feasibility  + dual_feasibility + complementary

#constraints_inv.append(coef[2:5] == 0)

# loss += cp.sum_squares(Pg_inv - Pg_scenarios)
# loss += cp.sum_squares(lambda_slack_inv - lambda_slack_scenarios)
# loss += cp.sum_squares(mu_min_inv - mu_min_scenarios)
# loss += cp.sum_squares(mu_max_inv - mu_max_scenarios)
# loss += cp.sum_squares(nu_min_inv - nu_min_scenarios)
# loss += cp.sum_squares(nu_max_inv - nu_max_scenarios)


loss *= 1 / num_scenarios

inv_prob = cp.Problem(cp.Minimize(loss), constraints_inv)
inv_prob.solve(solver=cp.MOSEK, verbose=True)

if inv_prob.status == 'optimal':
    print("Inferred cost coefficients: ", coef.value)
    print("True cost coefficients: ", cost_coeff_true)
    print("Optimal cost: ", inv_prob.value)
else:
    print("Inverse problem not optimal: ", inv_prob.status)


                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) May 01 03:19:53 PM: Your problem has 815 variables, 6060 constraints, and 0 parameters.
(CVXPY) May 01 03:19:54 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 01 03:19:54 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 01 03:19:54 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) May 01 03:19:54 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 01 03:19:54 PM: Compiling problem (target solver=MOSEK).

In [10]:
print("Inferred cost coefficients: ", coef.value)
print("True cost coefficients: ", cost_coeff_true)

# Inverse Optimization Variables
print("loss:\n", loss.value)
print("Pg_inv:\n", Pg_inv.value)
print("lambda_slack_inv:\n", lambda_slack_inv.value)
print("nu_max_inv:\n", nu_max_inv.value)
print("nu_min_inv:\n", nu_min_inv.value)
print("mu_min_inv:\n", mu_min_inv.value)
print("mu_max_inv:\n", mu_max_inv.value)
print("epsilon_line:\n", epsilon_line.value)
print("epsilon_gen:\n", epsilon_gen.value)


Inferred cost coefficients:  [ 7.924 23.272  0.004  0.004  0.004]
True cost coefficients:  [ 7.921 23.269  0.000  0.000  0.000]
loss:
 821.9469782880377
Pg_inv:
 [[280.670   0.007  -0.000  -0.000   0.000]
 [258.867  21.811  -0.000  -0.000   0.000]
 [237.056   0.000  -0.000  -0.000   0.000]
 [237.056   0.000  -0.000  -0.000   0.000]
 [231.043  11.147  -0.000  -0.000   0.000]
 [236.106   6.084  -0.000  -0.000   0.000]
 [271.363   0.000  -0.000  -0.000   0.000]
 [247.421  23.942  -0.000  -0.000   0.000]
 [219.896   0.000  -0.000  -0.000   0.000]
 [219.896   0.000  -0.000  -0.000   0.000]
 [230.022   0.000  -0.000  -0.000   0.000]
 [230.022   0.000  -0.000  -0.000   0.000]
 [266.221  14.033  -0.000  -0.000   0.000]
 [265.256  14.999  -0.000  -0.000   0.000]
 [223.479   0.000  -0.000  -0.000   0.000]
 [223.479   0.000  -0.000  -0.000   0.000]
 [280.435   2.964  -0.000  -0.000   0.000]
 [283.399   0.000  -0.000  -0.000   0.000]
 [244.674   0.000  -0.000  -0.000   0.000]
 [244.674   0.000  -0